In [1]:
# Query
# TODO: Search on Topic Signatures
# TODO: Search on Keyphrases
# TODO: Search on Noun-Verb Keyphrases
# TODO: Return k=5 highest

# Ranking
# TODO: Review DrQA
# TODO: Segment Articles into Paragraphs
# TODO: Segment Evidence Paragraphs into Sentences
# TODO: Re-rank according to TF-IDF
# TODO: Retain top k=10 Sentences as the evidence set

In [170]:
# DONE: Get Post
# DONE: Get Topic Signature
# DONE: Parse the full arguments, not the titles

import json

args = [json.loads(ln)["arguments"] for ln in open("./data/train_cmv_cleaned.jsonl")][0:10]
signatures = [json.loads(ln)["ratio_ranked_words"][0:3] for ln in open("./data/topic_signatures.jsonl")][0:10]

In [171]:
args

['I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would understand the distinction between natural and manmade. However I think unnatural and manma

In [172]:
signatures

[[['unnatural', 63.15551340835407, 5],
  ['natural', 56.79438773499214, 7],
  ['nature', 55.89856541018207, 7]],
 [['oil', 32.33425256235455, 3],
  ['wife', 16.659539989207133, 2],
  ['skeptical', 11.190158253867963, 1]],
 [['gun', 25.668442862567645, 4],
  ['ownership', 17.72733773278196, 2],
  ['train', 14.030417918105968, 2]],
 [['causation', 346.93649827873776, 28],
  ['mug', 198.73137356616772, 18],
  ['area', 99.9162981653185, 18]],
 [['philosophy', 237.63161457767546, 26],
  ['study', 159.11153174544233, 20],
  ['gain', 52.25974215691849, 8]],
 [['speech', 16.625745668964782, 2],
  ['Sony', 11.442826008990778, 1],
  ['cartoon', 11.260983839177868, 1]],
 [['IQ', 270.97422481972865, 24],
  ['offspring', 160.07290931675902, 15],
  ['twin', 101.06794709674443, 9]],
 [['post', 53.75049914919873, 10],
  ['controversial', 23.871155906926333, 3],
  ['educate', 21.900237518462063, 3]],
 [['small', 68.81630579014472, 10],
  ['car', 51.880609673125036, 8],
  ['drive', 38.54274352118682, 6]

In [174]:
arg_ = args[6]
sig_ = signatures[6]

print(arg_, sig_)

As an independent form of study philosophy doesnt seem to have any practical applications. What value does philosophy have in the modern age right now aside from contemplating things. Is it truly worth it to invest a significant amount of time and money studying this field? There do not seem to be any tangible applications or appreciable benefits from studying philosophy aside from personal growth and the expansion of ones intellectual perspective which I argue can be gained without studying philosophy in a rigorously academic manner. I often have read the argument that it is impossible to argue that philosophy is useless without using philosophy or something along those lines.I acknowledge this. Yes I am engaging in the use of philosophy right now at this very moment. However this does not provide an argument as to why it would be worthwhile to STUDY philosophy. What do you gain from studying philosophy that could not be gained from thoughtful introspection? Certainly important tools 

In [175]:
### DATABASE INIT ###
from retriver.elastic_db import ElasticDB

# TODOs: Store as Config
# Params
PORT = "http://localhost:9200"
INDEX = "wiki_evidence"

wiki_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX)

INFO:retriver.elastic_db:Connecting to http://localhost:9200


In [176]:
### ARGUMENT SENTENCE SEGMENTATION ###
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(arg_)

sents = []
for i in doc.sents:
    sents.append(i)

sents

[As an independent form of study philosophy doesnt seem to have any practical applications.,
 What value does philosophy have in the modern age right now aside from contemplating things.,
 Is it truly worth it to invest a significant amount of time and money studying this field?,
 There do not seem to be any tangible applications or appreciable benefits from studying philosophy aside from personal growth and the expansion of ones intellectual perspective which I argue can be gained without studying philosophy in a rigorously academic manner.,
 I often have read the argument that it is impossible to argue that philosophy is useless without using philosophy or something along those lines.,
 I acknowledge this.,
 Yes I am engaging in the use of philosophy right now at this very moment.,
 However this does not provide an argument as to why it would be worthwhile to STUDY philosophy.,
 What do you gain from studying philosophy that could not be gained from thoughtful introspection?,
 Certai

In [179]:
# TODOs: Use Chunking

terms_ = []
for i in sents:
    terms = set()
    for tok in i:
        if tok.pos_ == "NOUN" or tok.pos_ == "VERB":
            terms.add(tok)

    terms_.append(terms)

topics = [k[0] for k in sig_]
queries = []

# TODOs: Add Topic Signatures
for i in terms_:
    query = ", ".join([str(j).lower() for j in i])
    queries.append(query)

queries

['applications, form, philosophy, seem, study, have',
 'have, contemplating, philosophy, age, value, things',
 'field, money, amount, invest, time, studying',
 'gained, argue, philosophy, growth, applications, expansion, benefits, philosophy, studying, manner, ones, studying, seem, perspective',
 'philosophy, read, argument, philosophy, argue, using, lines',
 'acknowledge',
 'use, philosophy, engaging, moment',
 'study, provide, philosophy, argument',
 'introspection, gain, philosophy, studying, gained',
 'science, subset, originated, described, study, lack, argument, benefits, philosophy, gained, philosophy, method, tools, studying',
 'need, philosophy, become, study, scientist',
 'need, philosophy, life, encounter, circumstances, study, cultivate, principles, set, situations']

In [180]:
### DATABASE QUERY ###

passages = []
for _ in queries:
    passages.append((query, wiki_db.search(_, k=1)))

INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.008s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.005s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.004s]
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.004s]
I

In [181]:
passages

[('need, philosophy, life, encounter, circumstances, study, cultivate, principles, set, situations',
  ([{'_index': 'wiki_evidence',
     '_id': 'YxgrxYIBupgAEyuBfve9',
     '_score': 10.806194,
     '_source': {'id': '8036355',
      'title': 'Framework-specific modeling language',
      'text': 'Applications.'}}],
   ['8036355'])),
 ('need, philosophy, life, encounter, circumstances, study, cultivate, principles, set, situations',
  ([{'_index': 'wiki_evidence',
     '_id': 'NRgrxYIBupgAEyuBP_Pt',
     '_score': 8.909443,
     '_source': {'id': '8034831',
      'title': 'Peter Koslowski',
      'text': 'Peter Koslowski (2 October 1952 – 15 May 2012) was a professor of philosophy, especially philosophy of management and organisation and history of modern philosophy, at the Vrije Universiteit Amsterdam (VU University Amsterdam).'}}],
   ['8034831'])),
 ('need, philosophy, life, encounter, circumstances, study, cultivate, principles, set, situations',
  ([{'_index': 'wiki_evidence',
   

In [169]:
# TODOs: Segment Articles into Passages and Index in ElasticSearch
# TODOs: Update Build Elastic DB script, to segment

# COPY
def paragraphs(document):
    start = 0
    passages = []
    for token in document:
        if token.is_space and token.text.count("\n") > 1:
            yield document[start:token.i]
            start = token.i
    yield document[start:]
    #return passages.append(document[start:])

serach, article = articles[0]
article[0]

import json
art_ = nlp(article[0][0]["_source"]["text"])

passages = paragraphs(art_)
passages = [str(i) for i in paragraphs(art_)]
passages[0].split("\n")

['Engelbert (c. 1250 – 12 May 1331) was Abbot of the Benedictine monastery of Admont in Styria.',
 'Life.',
 'He was born of noble parents at Volkersdorf in Styria, and entered the monastery of Admont about 1267. Four years later he was sent to Prague to study grammar and logic. After devoting himself for two years to these studies he spent nine years at the University of Padua studying philosophy and theology.',
 'In 1297 he was elected Abbot of Admont, and after ruling thirty years he resigned this dignity when he was almost eighty years old, in order to spend the remainder of his life in prayer and study. Engelbert was one of the most learned men of his times, and there was scarcely any branch of knowledge to which his versatile pen did not contribute its share.',
 'Works.',
 'His literary productions include works on moral and dogmatic theology, philosophy, history, political science, Holy Scripture, the natural sciences, pedagogy, and music.',
 'The best known of Engelbert\'s work

In [1]:
import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  10
